Formação Cientista de Dados II - Prof. Fernando Amaral
Engenharia de Atributos

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder,  StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
dataset = pd.read_csv("credito4.csv", sep=";")
#visulizar
dataset.shape

In [ ]:
dataset.head()

In [ ]:
#separamos x de y
y = dataset['CLASSE']
X = dataset.iloc[:,0:7]

In [ ]:
#Tratamento de valores faltantes
X.isnull().sum()

In [ ]:
#preenche saldo atual pela mediana
mediana = X['SALDO_ATUAL'].median()
mediana

In [ ]:
X['SALDO_ATUAL'].fillna(mediana, inplace=True)
#checa resultado
X.isnull().sum()

In [ ]:
#verifica qual a "moda" do estado civil
agrupado = X.groupby(['ESTADOCIVIL']).size()
agrupado

In [ ]:
#substitui o estado civil pela moda
X['ESTADOCIVIL'].fillna('masculino solteiro', inplace=True)
#checa resultado
X.isnull().sum()

In [ ]:
#outliers, definimos como acima de 2 sd
desv = X['SALDO_ATUAL'].std()
desv

In [ ]:
#checamos se algum SALDO_ATUAL atende critério
X.loc[X['SALDO_ATUAL'] >=  2 * desv ,'SALDO_ATUAL'] 

In [ ]:
#vamos atualiar Saldo para mediana, calculamos
mediana =X['SALDO_ATUAL'].median()
mediana

In [ ]:
#atribumos
X.loc[X['SALDO_ATUAL'] >=  2 * desv, 'SALDO_ATUAL'] = mediana
#checamos se algum atende critério
X.loc[X['SALDO_ATUAL'] >=  2 * desv ] 

In [ ]:
#data binding
agrupado = X.groupby(['PROPOSITO']).size()
agrupado

In [ ]:
X.loc[X['PROPOSITO'] == 'Eletrodomésticos', 'PROPOSITO'] = 'outros'
X.loc[X['PROPOSITO'] == 'mobilia/equipamento', 'PROPOSITO'] = 'outros'
X.loc[X['PROPOSITO'] == 'educação', 'PROPOSITO'] = 'outros'
agrupado = X.groupby(['PROPOSITO']).size()
agrupado

In [ ]:
#extração de caracteristicas da data
#transforma coluna em data
X['DATA'] = pd.to_datetime(X['DATA'], format="%d/%m/%Y")

In [ ]:
X['DATA']

In [ ]:
#criamos 3 novos atributos
X['ANO'] = X['DATA'].dt.year
X['MES'] = X['DATA'].dt.month
X['DIASEMANA'] = X['DATA'].dt.day_name()

In [ ]:
X['DIASEMANA']

In [ ]:
#label encoder estado civil, proposito e dia semana
X['ESTADOCIVIL'].unique()

In [ ]:
X['PROPOSITO'].unique()

In [ ]:
X['DIASEMANA'].unique()

In [ ]:
labelencoder1 = LabelEncoder()
X['ESTADOCIVIL'] = labelencoder1.fit_transform(X['ESTADOCIVIL'])
X['PROPOSITO'] = labelencoder1.fit_transform(X['PROPOSITO'])
X['DIASEMANA'] = labelencoder1.fit_transform(X['DIASEMANA'])

In [ ]:
X.head()

In [ ]:
#one hot encodigo
#OUTROSPLANOSPGTO
outros = X['OUTROSPLANOSPGTO'].unique()
outros

In [ ]:
z = pd.get_dummies(X['OUTROSPLANOSPGTO'], prefix='OUTROS')

In [ ]:
z

In [ ]:
#padronização Z-score
#gera objeto numpy
sc = StandardScaler()
m = sc.fit_transform( X.iloc[:,0:3])

In [ ]:
m

In [ ]:
#concatena os dados
X = pd.concat([X,z,pd.DataFrame(m, columns=['SALDO_ATUAL_N','RESIDENCIADESDE_N','IDADE_N'])], axis=1)

In [ ]:
X

In [ ]:
#remover: saldo_atual e residenciadesde e idade foram normalizados
#outros planospagto foi feito onehot
#data foi extraida características
#OUTROS_banco por causa da dumy variable trap
X.drop(columns=['SALDO_ATUAL', 'RESIDENCIADESDE','IDADE','OUTROSPLANOSPGTO','DATA', 'OUTROS_banco'], inplace=True)

In [ ]:
X

In [ ]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X,y,test_size = 0.3,random_state = 0)
floresta = RandomForestClassifier(n_estimators = 100)
floresta.fit(X_treinamento, y_treinamento)
previsoes = floresta.predict(X_teste)
confusao = confusion_matrix(y_teste, previsoes)
taxa_acerto = accuracy_score(y_teste, previsoes)

In [ ]:
taxa_acerto